In [5]:
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.3-cp37-none-any.whl size=25853 sha256=d18b7a950ea1c4719bae52990b66273ecb6d4e01bc35f8b521a48c7e354ebfaf
  Stored in directory: C:\Users\User\AppData\Local\pip\Cache\wheels\16\84\1f\bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils


In [1]:
# import the necessary packages
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2
 
# defining the dictionary of the digit segments so we can identify them seperatly
# each digit on the thermostat i.e 0,1,2,3,4,5,6,7,8 and 9
DIGITS_LOOKUP = {
(1, 1, 1, 0, 1, 1, 1): 0,
(0, 0, 1, 0, 0, 1, 0): 1,
(1, 0, 1, 1, 1, 1, 0): 2,
(1, 0, 1, 1, 0, 1, 1): 3,
(0, 1, 1, 1, 0, 1, 0): 4,
(1, 1, 0, 1, 0, 1, 1): 5,
(1, 1, 0, 1, 1, 1, 1): 6,
(1, 0, 1, 0, 0, 1, 0): 7,
(1, 1, 1, 1, 1, 1, 1): 8,
(1, 1, 1, 1, 0, 1, 1): 9
}

In [5]:
# loading the example image
image = cv2.imread("example.jpg")

In [6]:
cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
# pre-process the image by resizing it, converting it into
# graycale, blurring it, and computing an edge map out of it
image = imutils.resize(image, height=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Can(blurred, 50, 200, 255)

In [7]:
cv2.imshow('image',edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [38]:
# find contours in the edge map, then sort them by their
# size in descending order
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
displayCnt = None
 
# looping over the contours
for c in cnts:
	# approximate the contour
	peri = cv2.arcLength(c, True)
	approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
	# if the contour has four vertices, then we have found
	# the thermostat display
	if len(approx) == 4:
		displayCnt = approx
		break

In [39]:
# extracting the thermostat display, apply a perspective transform
# to it
warped = four_point_transform(gray, displayCnt.reshape(4, 2))
output = four_point_transform(image, displayCnt.reshape(4, 2))

In [40]:
cv2.imshow('image',warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
cv2.imshow('image',output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [42]:
# threshold the warped image, then apply a series of morphological
# operations to cleanup the thresholded image
thresh = cv2.threshold(warped, 0, 255,
cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 5))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

In [43]:
cv2.imshow('image',thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [44]:
# find contours in the thresholded image, then initialize the
# digit contours lists
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
digitCnts = []
 
# loop over the digit area candidates
for c in cnts:
	# compute the bounding box of the contour
	(x, y, w, h) = cv2.boundingRect(c)
 
	# if the contour is sufficiently large, it must be a digit
	if w >= 15 and (h >= 30 and h <= 40):
		digitCnts.append(c)

In [46]:
# sort the contours from left-to-right, then initialize the
# actual digits themselves
digitCnts = contours.sort_contours(digitCnts,
	method="left-to-right")[0]
digits = []

In [47]:
# loop over each of the digits
for c in digitCnts:
	# extract the digit ROI
	(x, y, w, h) = cv2.boundingRect(c)
	roi = thresh[y:y + h, x:x + w]
 
	# compute the width and height of each of the 7 segments
	# we are going to examine
	(roiH, roiW) = roi.shape
	(dW, dH) = (int(roiW * 0.25), int(roiH * 0.15))
	dHC = int(roiH * 0.05)
 
	# define the set of 7 segments
	segments = [
		((0, 0), (w, dH)),	# top
		((0, 0), (dW, h // 2)),	# top-left
		((w - dW, 0), (w, h // 2)),	# top-right
		((0, (h // 2) - dHC) , (w, (h // 2) + dHC)), # center
		((0, h // 2), (dW, h)),	# bottom-left
		((w - dW, h // 2), (w, h)),	# bottom-right
		((0, h - dH), (w, h))	# bottom
	]
	on = [0] * len(segments)

In [48]:
# loop over the segments
for (i, ((xA, yA), (xB, yB))) in enumerate(segments):
	# extract the segment ROI, count the total number of
	# thresholded pixels in the segment, and then compute
	# the area of the segment
	segROI = roi[yA:yB, xA:xB]
	total = cv2.countNonZero(segROI)
	area = (xB - xA) * (yB - yA)

	# if the total number of non-zero pixels is greater than
	# 50% of the area, mark the segment as "on"
	if total / float(area) > 0.5:
		on[i]= 1
 
# lookup the digit and draw it on the image
digit = DIGITS_LOOKUP[tuple(on)]
digits.append(digit)
cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 1)
cv2.putText(output, str(digit), (x - 10, y - 10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)

array([[[ 42,  61,  82],
        [ 64,  81, 100],
        [ 64,  82,  99],
        ...,
        [ 69,  89, 107],
        [ 70,  90, 108],
        [ 69,  89, 107]],

       [[ 74,  96, 114],
        [ 75,  92, 111],
        [ 76,  94, 111],
        ...,
        [ 67,  87, 105],
        [ 67,  87, 105],
        [ 67,  87, 105]],

       [[ 74,  96, 114],
        [ 74,  91, 110],
        [ 81,  99, 116],
        ...,
        [ 66,  86, 104],
        [ 65,  85, 103],
        [ 67,  87, 105]],

       ...,

       [[158, 172, 190],
        [159, 173, 191],
        [156, 172, 189],
        ...,
        [143, 159, 176],
        [143, 159, 176],
        [142, 158, 175]],

       [[154, 168, 186],
        [155, 169, 187],
        [156, 172, 189],
        ...,
        [145, 161, 178],
        [145, 161, 178],
        [144, 160, 177]],

       [[156, 170, 188],
        [156, 170, 188],
        [156, 172, 189],
        ...,
        [144, 160, 177],
        [144, 160, 177],
        [143, 159, 176]]

In [49]:
# display the digits into a window
# print(r"{}{}.{} \u00b0C".format(*digits))
cv2.imshow("Input", image)
cv2.imshow("Output", output)
cv2.waitKey(0)

-1